# 03: Spectrum Analysis

This notebook covers FFT-based spectrum analysis techniques.

## Learning Objectives
- Understand the FFT and its relationship to frequency content
- Apply window functions to reduce spectral leakage
- Create waterfall/spectrogram displays
- Use R4W's analysis tools

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from r4w_python import plot_spectrum, plot_waterfall, compute_spectrum, compute_waterfall

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## FFT Fundamentals

The Fast Fourier Transform (FFT) converts time-domain samples to frequency-domain:

$$X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j2\pi kn/N}$$

Key relationships:
- Frequency resolution: $\Delta f = f_s / N$
- Maximum frequency: $f_{max} = f_s / 2$ (Nyquist)

In [ ]:
# Generate a single tone
sample_rate = 48000
duration = 0.1
frequency = 5000  # Hz

t = np.arange(0, duration, 1/sample_rate)
tone = np.exp(2j * np.pi * frequency * t)

print(f"Samples: {len(tone)}")
print(f"Frequency resolution: {sample_rate/len(tone):.1f} Hz")

In [ ]:
# Compute and plot spectrum
freqs, power = compute_spectrum(tone, fft_size=1024)

fig, ax = plt.subplots(figsize=(10, 4))
plot_spectrum(power, freqs * sample_rate, sample_rate, title=f"{frequency} Hz Tone Spectrum", ax=ax)
plt.show()

## Window Functions

The FFT assumes the signal is periodic. Non-integer cycles cause **spectral leakage**.

Window functions taper the edges to reduce this artifact:
- **Rectangular** (no window): Best resolution, worst leakage
- **Hann**: Good general purpose
- **Blackman**: Low sidelobes, wider mainlobe

In [ ]:
# Create windows
n = 256
windows = {
    'Rectangular': np.ones(n),
    'Hann': np.hanning(n),
    'Blackman': np.blackman(n),
}

# Plot window shapes
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Time domain
for name, win in windows.items():
    axes[0].plot(win, label=name)
axes[0].set_title('Window Functions')
axes[0].set_xlabel('Sample')
axes[0].legend()

# Frequency response
for name, win in windows.items():
    spectrum = np.fft.fftshift(np.fft.fft(win, 4096))
    power = 20 * np.log10(np.abs(spectrum) / np.max(np.abs(spectrum)) + 1e-10)
    freq = np.linspace(-0.5, 0.5, len(power))
    axes[1].plot(freq, power, label=name)

axes[1].set_title('Window Frequency Response')
axes[1].set_xlabel('Normalized Frequency')
axes[1].set_ylabel('dB')
axes[1].set_ylim(-100, 5)
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Demonstrate spectral leakage
# Non-integer cycles cause leakage
f_exact = 1000  # Exactly 4 cycles in 256 samples at 64000 Hz -> integer
f_offset = 1050  # Non-integer cycles

n = 256
t = np.arange(n) / sample_rate

sig_exact = np.exp(2j * np.pi * f_exact * t)
sig_offset = np.exp(2j * np.pi * f_offset * t)

fig, axes = plt.subplots(2, 1, figsize=(12, 6))

for sig, title, ax in [(sig_exact, f'{f_exact} Hz (Integer cycles)', axes[0]),
                        (sig_offset, f'{f_offset} Hz (Non-integer cycles)', axes[1])]:
    for name, win in [('Rectangular', np.ones(n)), ('Hann', np.hanning(n))]:
        freqs, power = compute_spectrum(sig, fft_size=n, window=name.lower())
        ax.plot(freqs * sample_rate / 1000, power, label=name)
    ax.set_title(title)
    ax.set_xlabel('Frequency (kHz)')
    ax.set_ylabel('Power (dB)')
    ax.legend()
    ax.set_xlim(-5, 5)

plt.tight_layout()
plt.show()

## Waterfall Displays (Spectrograms)

A waterfall shows how the spectrum changes over time:
- X-axis: Frequency
- Y-axis: Time
- Color: Power

In [ ]:
# Generate a chirp (frequency sweep)
duration = 0.1
t = np.arange(0, duration, 1/sample_rate)
f_start = 1000
f_end = 10000

# Linear chirp
chirp = np.exp(2j * np.pi * (f_start * t + (f_end - f_start) / (2 * duration) * t**2))

print(f"Generated chirp from {f_start} Hz to {f_end} Hz")

In [ ]:
# Compute and plot waterfall
waterfall = compute_waterfall(chirp, fft_size=256, hop_size=64)

fig, ax = plt.subplots(figsize=(12, 6))
plot_waterfall(waterfall, sample_rate=sample_rate, fft_size=256, hop_size=64,
               title='Linear Chirp Spectrogram', ax=ax)
plt.show()

## Analyzing Multiple Signals

In [ ]:
# Multiple tones with different powers
t = np.arange(0, 0.1, 1/sample_rate)
multi_tone = (
    1.0 * np.exp(2j * np.pi * 2000 * t) +  # Strong
    0.3 * np.exp(2j * np.pi * 5000 * t) +  # Medium
    0.1 * np.exp(2j * np.pi * 8000 * t)    # Weak
)

freqs, power = compute_spectrum(multi_tone, fft_size=2048)

fig, ax = plt.subplots(figsize=(12, 4))
plot_spectrum(power, freqs * sample_rate, sample_rate, 
              title='Multi-Tone Spectrum (2, 5, 8 kHz)', ax=ax)
plt.show()

## Exercises

1. **Resolution**: What FFT size is needed to resolve two tones 100 Hz apart at 48 kHz sample rate?

2. **Dynamic range**: Add a weak tone (-40 dB) near a strong tone. Can you see it with different windows?

3. **Hop size**: How does waterfall hop size affect time resolution vs frequency resolution?

In [ ]:
# Your code here
